In [1]:
import dataset
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#explain dataframe
tropes = dataset.TVTropesDataset.from_csv_files("TVTropesData")
tropes.lit_tropes
tropes.lit_tropes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679618 entries, 0 to 679617
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  679618 non-null  int64 
 1   Title       679618 non-null  object
 2   Trope       679618 non-null  object
 3   Example     662150 non-null  object
 4   trope_id    679618 non-null  object
 5   title_id    679618 non-null  object
dtypes: int64(1), object(5)
memory usage: 31.1+ MB


In [8]:
# statistics about Example column text data (avg number of chars etc.)
tropes.lit_tropes["Example"].describe()

count                                                662150
unique                                               554323
top        Fighting Fantasy, a wildly popular British se...
freq                                                     68
Name: Example, dtype: object

In [11]:
tropes.lit_tropes["Example"].value_counts()

Example
Fighting Fantasy, a wildly popular British series that includes simple Tabletop RPG elements, with dice as randomizers.\n The Warlock of Firetop Mountain by Steve Jackson and Ian Livingstone, the first installment and the one that started it all.\n The Citadel of Chaos by Steve Jackson, second installment of the series.\n The Forest of Doom by Ian Livingstone, third installment of the series.\n Starship Traveller by Steve Jackson, fourth installment of the series and first foray into Sci-Fi, heavily inspired by Star Trek.\n City of Thieves by Ian Livingstone, 5th installment of the series.\n Deathtrap Dungeon by Ian Livingstone, 6th installment of the series.\n Island of the Lizard King by Ian Livingstone, 7th installment of the series.\n Scorpion Swamp by Steve Jackson,noteNot the same Jackson who co-wrote Firetop Mountain and wrote Citadel, but the US game developer 8th installment of the series. Features unusual, multiple alignment-based paths and endings.\n Caverns of the S

In [15]:
# check sample of non unique rows
tropes.lit_tropes.nunique()

Unnamed: 0    679618
Title          15495
Trope          27229
Example       554323
trope_id       27229
title_id       15495
dtype: int64

In [ ]:
# check sample of unique rows
tropes.lit_tropes.unique()

KeyboardInterrupt: 